<img src='https://upload.wikimedia.org/wikipedia/commons/thumb/a/aa/Logo_DuocUC.svg/2560px-Logo_DuocUC.svg.png' width=50%, height=20%>

# INTRODUCCION

Para este ejemplo, usaremos un grupo de datos de estudiantes ya graduados y trataremos de probar si su grado afecta su salario.

Sea μ el salario medio anual de los graduados.

Supongamos que tenemos 6 categorías de materias de estudio:= Matemáticas y Estadística, Economía y Finanzas, Ciencias Ambientales, Ciencias Políticas, Ciencias Sociales y Biología.

Por lo tanto, hay 6 niveles o grupos de este único factor que afecta la media del salario anual de los graduados.

La idea básica detrás de un ANOVA  es tomar muestras aleatorias independientes de cada grupo y luego calcular las medias de muestra para cada grupo. Después de eso, comparar la variación de las medias muestrales entre los grupos con la variación dentro de los grupos. Finalmente, tomar una decisión basada en una prueba estadística, ya sea que las medias de los grupos sean todas iguales o no.

In [1]:
import pandas as pd
import random

# leer archivo ('students.csv')
name = '../data/Students.csv'
student_df = pd.read_csv(name, sep=",")

# filtrar los estudiantes graduados
graduated_student_df = student_df[student_df['graduated'] == 1]

# muestra random de 500 estudiantes
unique_student_id = list(graduated_student_df['stud.id'].unique())
random.seed(30) # establecemos una semilla para extraer siempre la misma muestra
sample_student_id = random.sample(unique_student_id, 500)
sample_df = graduated_student_df[graduated_student_df['stud.id'].isin(sample_student_id)].reset_index(drop=True)

# dos variables de interes
sample_df = sample_df[['major', 'salary']]
groups = sample_df.groupby('major').count().reset_index()
#
groups

,major,salary
0,Biology,87
1,Economics and Finance,94
2,Environmental Sciences,97
3,Mathematics and Statistics,75
4,Political Science,84
5,Social Sciences,63


# Planteamiento de Hipótesis

Recordemos el planteamiento de la hipótesis:


H₀: μ₁= μ₂ = μ₃ = … = μ𝒸


H₁: No todos los μᵢ son iguales, donde i = 1, 2, 3, …

Grados de libertad (ya calculados) = α = 0.05

Regla de decisión: se rechaza H₀, cuando F > Fα

In [2]:
import numpy as np 
# Scipy tiene un paquete o módulo scipy.stats que contiene una gran cantidad de funciones estadísticas.
import scipy.stats as stats
import matplotlib.pyplot as plt


In [3]:
# Crea la tabla principal de ANOVA
data = [['Between Groups', '', '', '', '', '', ''], ['Within Groups', '', '', '', '', '', ''], ['Total', '', '', '', '', '', '']] 
anova_table = pd.DataFrame(data, columns = ['Source of Variation', 'SS', 'df', 'MS', 'F', 'P-value', 'F crit']) 
anova_table.set_index('Source of Variation', inplace = True)

# calcula ENTRE los grupos
x_bar = sample_df['salary'].mean()
SSTR = sample_df.groupby('major').count() * (sample_df.groupby('major').mean() - x_bar)**2
anova_table['SS']['Between Groups'] = SSTR['salary'].sum()

# calcula EN los grupos
SSE = (sample_df.groupby('major').count() - 1) * sample_df.groupby('major').std()**2
anova_table['SS']['Within Groups'] = SSE['salary'].sum()

# calcula el total
SSTR = SSTR['salary'].sum() + SSE['salary'].sum()
anova_table['SS']['Total'] = SSTR

# Actualiza los grados de libertad
anova_table['df']['Between Groups'] = sample_df['major'].nunique() - 1
anova_table['df']['Within Groups'] = sample_df.shape[0] - sample_df['major'].nunique()
anova_table['df']['Total'] = sample_df.shape[0] - 1

# calcula la media de los cuadrados
anova_table['MS'] = anova_table['SS'] / anova_table['df']

# calcula F 
F = anova_table['MS']['Between Groups'] / anova_table['MS']['Within Groups']
anova_table['F']['Between Groups'] = F

# p-value
anova_table['P-value']['Between Groups'] = 1 - stats.f.cdf(F, anova_table['df']['Between Groups'], anova_table['df']['Within Groups'])

# F critico 
alpha = 0.05
# 
tail_hypothesis_type = "two-tailed"
if tail_hypothesis_type == "two-tailed":
    alpha /= 2
anova_table['F crit']['Between Groups'] = stats.f.ppf(1-alpha, anova_table['df']['Between Groups'], anova_table['df']['Within Groups'])

# Mostrar tabla ANOVA
anova_table

/tmp/ipykernel_76524/817107024.py:9: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  anova_table['SS']['Between Groups'] = SSTR['salary'].sum()
/tmp/ipykernel_76524/817107024.py:13: FutureWarning: ChainedAssignmentError: behaviour will change 

,SS,df,MS,F,P-value,F crit
Source of Variation,,,,,,
Between Groups,24184075535.074078,5,4836815107.014815,77.408037,0.0,2.592178
Within Groups,30867423591.219269,494,62484663.14012,,,
Total,55051499126.29335,499,110323645.543674,,,


In [4]:
    
# Resultado final
print("\n--------------------------------------------------------------------------------------")
print("El enfoque del valor crítico para la prueba de hipótesis en la regla de decisión")
conclusion = "No se pudo rechazar la hipótesis nula."
if anova_table['F']['Between Groups'] > anova_table['F crit']['Between Groups']:
    conclusion = "Se rechaza la hipótesis nula."
print("F-score es:", anova_table['F']['Between Groups'], " y valor critico es:", anova_table['F crit']['Between Groups'])
print(conclusion)


--------------------------------------------------------------------------------------
El enfoque del valor crítico para la prueba de hipótesis en la regla de decisión
F-score es: 77.40803685167356  y valor critico es: 2.592178494905
Se rechaza la hipótesis nula.


# Conclusión: 
Hay suficiente evidencia de que no todos los salarios promedio son iguales para los egresados de las diferentes materias de estudio, con un 5% de nivel de significancia.

EJERCICIO : INTENTA PROBAR CON DIFERENTES GRADOS DE LIBERTAD (según el valor encontrado de F) Y VE QUE PASA CON LA HIPOTESIS....
